In [123]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.porter import *
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [145]:
df=pd.read_csv("/content/drive/MyDrive/labels.csv")
df.head()

,Unnamed: 0,image_name,text_ocr,text_corrected,overall_sentiment
0,0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,very_positive
1,1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,very_positive
2,2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,positive
3,3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,positive
4,4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,neutral


In [146]:
df.shape

(6992, 5)

In [147]:
from numpy import NaN
df[df['text_corrected'] == NaN]

,Unnamed: 0,image_name,text_ocr,text_corrected,overall_sentiment


In [148]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6992 entries, 0 to 6991
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         6992 non-null   int64 
 1   image_name         6992 non-null   object
 2   text_ocr           6831 non-null   object
 3   text_corrected     6987 non-null   object
 4   overall_sentiment  6992 non-null   object
dtypes: int64(1), object(4)
memory usage: 273.2+ KB


In [149]:
df.describe()

,Unnamed: 0
count,6992.000000
mean,3495.500000
std,2018.560873
min,0.000000
25%,1747.750000
50%,3495.500000
75%,5243.250000
max,6991.000000


In [150]:
df.isnull().sum()

Unnamed: 0             0
image_name             0
text_ocr             161
text_corrected         5
overall_sentiment      0
dtype: int64

In [151]:
# df['text_corrected'].dropna(axis = 0, how ='all', inplace=True)
# df['text_corrected']=df['text_corrected'].dropna()
# df['text_corrected']
# df.isnull().sum()
df['text_corrected'] = df['text_corrected'].fillna(" ")

In [152]:
df.isnull().sum()

Unnamed: 0             0
image_name             0
text_ocr             161
text_corrected         0
overall_sentiment      0
dtype: int64

In [153]:
df.shape

(6992, 5)

In [154]:
# sentiments=df['overall_sentiment'].value_counts()
# sentiments

In [155]:
# sentiments=df['overall_sentiment'].value_counts().sort_index().plot(kind='bar',title="Count of sentiments", figsize=(10,5))
# sentiments


In [156]:
# pd.value_counts(np.array(words))
from collections import Counter
from itertools import chain

# split words into lists
v = df['text_corrected'].str.split().tolist() # [s.split() for s in df['Col2'].tolist()]
# compute global word frequency
c = Counter(chain.from_iterable(v))
# filter, join, and re-assign
df['text_corrected'] = [' '.join([j for j in i if c[j] > 1]) for i in v]

In [157]:

stemmer=PorterStemmer()
cleaned_text=[]
text_list=df['text_corrected']
text_list
for word in text_list:
  words=re.sub('[^a-zA-Z]'," ",str(word))
  words=re.sub(r"RT[\s]+"," ",str(words))
  words=re.sub(r"http\S|www\S|https\S+","",str(words))
  words=words.lower()
  words=nltk.word_tokenize(str(words))
  words=[word for word in words if word not in set(stopwords.words('english'))]
  words=[stemmer.stem(word) for word in words]
  words=" ".join(words)
  # print(words)
  cleaned_text.append(words)
  # break
  

In [158]:
cleaned_text

['look friend play year challeng facebook imgflip com',
 'best less year pm jan india',
 'follow follow saw everyon post vs pic mine pm jan retweet like',
 'year challeng sweet edit',
 'year challeng filter hilari year challeng meme year',
 'get car meet peopl internet memecent com order internet get car',
 'year challeng b wow',
 'year challeng emot boredpanda com',
 'fornit die month minecraft never die year proud',
 'facebook year simpl meme jan',
 'probabl first man year challeng chennai',
 'state white vs boredpanda com',
 'facebook year challeng kind went year futur instead back',
 'movi jamaica meme ami food floor meme',
 'anti vaxx kid see someon year challeng',
 'get year challeng time someth',
 'read messag sent actual famili member instead friend drunk',
 'look meme realiz lost game want die',
 'year challeng year',
 'spoiler aveng infin thano silenc ig',
 'rememb feel old',
 'hulk right charact',
 'aveng movi work forc sound great would probabl',
 'got ta jar awesom pictur 

In [159]:
freq = pd.value_counts(np.array(cleaned_text))
freq

                                                                                                                  102
like                                                                                                                6
name                                                                                                                6
memegener net                                                                                                       5
html head meta content text html charset utf http equiv content type style type text css ol margin pad tabl td      5
                                                                                                                 ... 
kiss liam fan                                                                                                       1
liam drink beer like boss quickmem com                                                                              1
know lost phone imgflip com                             

13

In [125]:

unigram = []
for i in cleaned_text:
  x = i.split(" ")
  unigram.append(x)


In [128]:

pd.value_counts(np.array(unigram[0]))

look         1
friend       1
lightyear    1
sohalikut    1
trend        1
play         1
year         1
challeng     1
facebook     1
imgflip      1
com          1
dtype: int64

In [160]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(cleaned_text).toarray()
print(X)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [161]:
# y=pd.get_dummies(df["overall_sentiment"])
# y=y.values
# print(y)

In [162]:

lee = preprocessing.LabelEncoder()
y=df["overall_sentiment"]
y=lee.fit_transform(y)
y


array([4, 4, 2, ..., 2, 4, 2])

In [163]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [164]:
# classifier=MultinomialNB()
# classifier.fit(X_train,y_train)

In [165]:
# from sklearn.svm import LinearSVC
# classifier=LinearSVC()
# classifier.fit(X_train,y_train)

In [166]:
# from sklearn.naive_bayes import GaussianNB
# classifier=GaussianNB()

# classifier.fit(X_train,y_train)

In [167]:
# from sklearn.svm import SVC
# classifier=SVC()
# classifier.fit(X_train,y_train)

In [168]:
from sklearn.neighbors import KNeighborsClassifier
classifier=KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train,y_train)

KNeighborsClassifier()

In [171]:
from sklearn.linear_model import LogisticRegression
classifier=LogisticRegression()
classifier.fit(X_train,y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [172]:
y_pred=classifier.predict(X_test)
y_pred

array([2, 2, 4, ..., 2, 2, 2])

In [173]:
print(accuracy_score(y_test,y_pred))

0.40886347390993566


In [27]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)

[[  1  32   1   0  58]
 [  6 185  21   0 228]
 [  7 273  36   0 310]
 [  0  14   2   0  13]
 [  2 101  13   0  96]]
